In [211]:
def random_prime(a, b):
    while True:
        p = ZZ.random_element(2**k, 2**(k+1))
        if p.is_prime() == True:
            return p
 

class Shamir():
    
    def __init__(self, t, n):
        self._t = t
        self._n = n

        
    def GC(self, k):
        ## (a) pick a prime p
        p = random_prime(2**k, 2**(k+1))
        assert p > self._n
        
        ## (b) pick uniformly at random n distinct non-zero elements alpha from Zp*
        Zp = Zmod(p)
        alpha = []
        for i in range(self._n):
            alpha.append(Zp.random_element())
        
        return (p, alpha)
        # return x=(p, alpha)
        
        
    def DS(self, p, alpha, s):
        ## generate t-1 uniformly random elements a=(a_1,...,a_(t-1)) from Zp
        Zp = Zmod(p)
        a = [s]
        for i in range(self._t - 1):
            a.append(Zp.random_element()) 
            
        # build the polynomial a(x)=s+a_1*x+...+a_(t-1)*x^(t-1) from Zp[x;t-1]
        F = FiniteField(p)
        P = F['x']
        ax = P(a)
        
        # generate shares sigma_i
        shares = []
        for i in range(self._n):
            sigma_i = ax(alpha[i]) % p
            shares.append([alpha[i], sigma_i])
        return shares
        
        
    def SC(self, p, shares):
        assert len(shares) == self._t
        F = FiniteField(p)
        P = F['x']
        print (P.lagrange_polynomial(shares))
        
    def SG(self, sigma_i, p, H):
        r_i = randint(0, H)
        R = Integers(p)
        return R(sigma_i[0] * sigma_i[1] + r_i)

t = 3
n = 20
t_new = 8
k = 99

s = 100

shaTSS = Shamir(t, n)
p, alpha = shaTSS.GC(k)
shares = shaTSS.DS(p, alpha, s)
#print (shares)

sigma_c = 2**(-n)
appr = ((t+t_new)**(1/2)*2**((t+t_new)/2))#.n()
g_cvp = log(appr, 2)#.n()
sigma_F = t_new/t/k*(log(sigma_c**(-1/t_new)*n*t, 2) + g_cvp + 1)#.n()
alpha_noise = (1 - (1+sigma_F) / (t_new/t))#.n()

H = floor(p**alpha_noise / 2)#.n()
#H = p**1262 # test
 
shares_new = []
for share in shares:
    shares_new.append(shaTSS.SG(share, p, H))
    
M = []
for i in range(t+t_new):
    row = [0] * (t+t_new)
    if i < t_new:
        row[i] = p
    else:
        row[i] = H / p
        for j in range(t_new):
            row[j] = shares[j][0] ** (i-t_new+1)
    M.append(row)

M = matrix(QQ, M)
target = shares_new[:t_new] + [0]*t

def Babai_CVP(Lattice, w):
    L = Lattice.LLL()
    vt = L.solve_left(vector(QQ, w))
    v = vector(ZZ, [0]*len(L[0]))
    for t, vi in zip(vt, L):
        v += round(t) * vi
    return v

c = Babai_CVP(M, target)
print (c)

R = Integers(p)
print ((p/H)*c[t_new])
print (type((p/H)*c[t_new]))
recovered_s = К((p/H)*c[t_new]), p)
print (recovered_s)
assert s == recovered_s


#shaTSS.SC(p, shares[:3] + [[123, 123]])
#shaTSS.SC(p, shares[:4])

    


(342792569668631717017631272391, 212624346024060363221637248740, 565944495746144151435011979091, 10600780437294980876064881273, 179514797320481832763568322334, 86068288184420728374845326501, 488610377359199089342888510850, 167886825195428276189909486188, 2525538946673400/974039119929738287219778818719, -6532597984910616396440127954910565806617438/974039119929738287219778818719, 6802738444700965753037528707819635749196858/974039119929738287219778818719)
100
<class 'sage.rings.rational.Rational'>
100
